In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# **Without Sentiment**

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf

In [3]:
df_stock_tesla = yf.download('TSLA', start='2023-01-01', end='2024-02-14')

[*********************100%%**********************]  1 of 1 completed


In [4]:
df_stock_tesla

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,118.470001,118.800003,104.639999,108.099998,108.099998,231402800
2023-01-04,109.110001,114.589996,107.519997,113.639999,113.639999,180389000
2023-01-05,110.510002,111.750000,107.160004,110.339996,110.339996,157986300
2023-01-06,103.000000,114.389999,101.809998,113.059998,113.059998,220911100
2023-01-09,118.959999,123.519997,117.110001,119.769997,119.769997,190284000
...,...,...,...,...,...,...
2024-02-07,188.179993,189.789993,182.679993,187.580002,187.580002,111535200
2024-02-08,189.000000,191.619995,185.580002,189.559998,189.559998,83034000
2024-02-09,190.179993,194.119995,189.479996,193.570007,193.570007,84476300


In [5]:
df_stock_tesla = pd.DataFrame(df_stock_tesla)

# Shift 'Open' column by one row to get the next day's open
next_day_open = df_stock_tesla['Open'].shift(-1)

# Add the next day's open as a new column
df_stock_tesla['Next Day Open'] = next_day_open
df_stock_tesla.dropna(inplace=True)
df_stock_tesla.head(5)

,Open,High,Low,Close,Adj Close,Volume,Next Day Open
Date,,,,,,,
2023-01-03,118.470001,118.800003,104.639999,108.099998,108.099998,231402800,109.110001
2023-01-04,109.110001,114.589996,107.519997,113.639999,113.639999,180389000,110.510002
2023-01-05,110.510002,111.750000,107.160004,110.339996,110.339996,157986300,103.000000
2023-01-06,103.000000,114.389999,101.809998,113.059998,113.059998,220911100,118.959999
2023-01-09,118.959999,123.519997,117.110001,119.769997,119.769997,190284000,121.070000


In [6]:
# df_stock_tesla = df_stock_tesla.iloc[::-1]

In [7]:
df_stock_tesla.drop(columns=['Adj Close'], inplace=True)

## Splitting the columns for target variable

In [8]:
X= df_stock_tesla.drop(columns=['Next Day Open'])
y=df_stock_tesla['Next Day Open']

## Scaling

In [9]:
# Importing libraries
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [10]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Define the column transformer
ct = make_column_transformer(
    (make_pipeline(KNNImputer(), StandardScaler()), slice(0, df_stock_tesla.shape[1])),  # Apply to all columns
    remainder="passthrough"
)

# Fit and transform the data
scaled_data_X = ct.fit_transform(X)
scaled_X = pd.DataFrame(scaled_data_X, columns=X.columns)

scaler_y = StandardScaler()
scaled_y = scaler_y.fit_transform(y.values.reshape(-1, 1))

## Spliting the data into Train,Test

In [11]:
# Determine the index to split the data linearly
split_index = int(len(X) * 0.85)

# Split the data into training and testing sets
X_train = scaled_data_X[:split_index]
X_test = scaled_data_X[split_index:]
y_train = scaled_y[:split_index]
y_test = scaled_y[split_index:]

## Modeling

In [12]:
# Importing librarues
from sklearn.linear_model import (LinearRegression, ElasticNetCV, RidgeCV)
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

In [13]:
# Collecting the different type of estimators to select best regression model
estimators = {
    "lr": LinearRegression(),
    "lassocv": LassoCV(alphas=np.logspace(-5, -1, 100)),
    "ridge": RidgeCV(alphas=np.logspace(-5, -1, 100)),
    "elastic": ElasticNetCV(alphas=np.logspace(-5, 1, 100), l1_ratio=[.1, .5, .7, .9, .95, .99, 1]),
}

In [14]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
import pandas as pd

# Initialize an empty dictionary to store results
# results_dict = {'Estimator': [], 'R2 Mean': [], 'R2 Std': [], 'RMSE Mean': [], 'RMSE Std': [], 'NRMSE Mean': [], 'NRMSE Std': []}
results_dict = {'Estimator': [], 'R2 Mean': [], 'RMSE Mean': [], 'NRMSE Mean': []}

# Iterate over each estimator
for name, estimator in estimators.items():
    print(name)

    # Perform cross-validation
    cv_result = cross_validate(estimator, X_train, y_train, scoring=('r2', 'neg_root_mean_squared_error'), return_estimator=True, cv=10, verbose=2)

    # Extract R2 scores and RMSE scores
    r2_scores = cv_result['test_r2']
    rmse_scores = -cv_result['test_neg_root_mean_squared_error']

    # Calculate mean and standard deviation of scores
    r2_mean, r2_std = np.mean(r2_scores), np.std(r2_scores)
    rmse_mean, rmse_std = np.mean(rmse_scores), np.std(rmse_scores)

    # Calculate NRMSE
    y_true = scaled_y
    y_preds = -cv_result['test_neg_root_mean_squared_error']  # Negative RMSE from cross-validation
    mse = y_preds ** 2  # Convert negative RMSE to MSE
    nrmse = sqrt(mse.mean()) / (y_true.max() - y_true.min())  # NRMSE calculation

    # Append results to the dictionary
    results_dict['Estimator'].append(name)
    results_dict['R2 Mean'].append(round(r2_mean, 3))
    #results_dict['R2 Std'].append(round(r2_std, 3))
    results_dict['RMSE Mean'].append(round(rmse_mean, 3))
    #results_dict['RMSE Std'].append(round(rmse_std, 3))
    results_dict['NRMSE Mean'].append(round(nrmse, 3))


# Create DataFrame from the results dictionary
results_df = pd.DataFrame(results_dict)

# Print the DataFrame
print(results_df)


lr
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
lassocv
[CV] END .................................................... total time=   0.3s
[CV] END .................................................... total time=   0.5s
[CV] END ........

In [15]:
pip install xgboost

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [17]:
estimators = {
    "RandomForest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 5, 10, 15]
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor(),
        "params": {
            "max_depth": [None, 5, 10, 15]
        }
    },
    "SVR": {
        "model": SVR(),
        "params": {
            "kernel": ['linear', 'rbf'],
            "C": [0.1, 1, 10]
        }
    },
    "XGBoost": {
        "model": XGBRegressor(),
        "params": {
            "max_depth": [3, 5, 7],
            "learning_rate": [0.1, 0.01, 0.001],
            "n_estimators": [50, 100, 200]
        }
    }
}

In [18]:
for name, estimator in estimators.items():
    print(name)
    grid_search = GridSearchCV(estimator["model"], estimator["params"], scoring=('r2', 'neg_root_mean_squared_error'), cv=10, refit='r2', verbose=2)
    grid_search.fit(scaled_X, scaled_y)

    r2_scores = grid_search.cv_results_['mean_test_r2']
    rmse_scores = -grid_search.cv_results_['mean_test_neg_root_mean_squared_error']

    # Calculate NRMSE
    y_true = scaled_y  # Assuming scaled_y is the true target variable
    y_preds = -grid_search.cv_results_['mean_test_neg_root_mean_squared_error']  # Negative RMSE from cross-validation
    mse = y_preds ** 2  # Convert negative RMSE to MSE
    nrmse = sqrt(mse.mean()) / (y_true.max() - y_true.min())  # NRMSE calculation

    results_dict['Estimator'].append(name)
    results_dict['R2 Mean'].append(round(np.mean(r2_scores), 3))
    #results_dict['R2 Std'].append(round(np.std(r2_scores), 3))
    results_dict['RMSE Mean'].append(round(np.mean(rmse_scores), 3))
    #results_dict['RMSE Std'].append(round(np.std(rmse_scores), 3))
    results_dict['NRMSE Mean'].append(round(nrmse, 3))



RandomForest
Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.1s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_d

In [19]:
results_df_without_sentiment = pd.DataFrame(results_dict)
print(results_df_without_sentiment)

      Estimator  R2 Mean  RMSE Mean  NRMSE Mean
0            lr    0.856      0.104       0.022
1       lassocv    0.855      0.104       0.022
2         ridge    0.857      0.104       0.022
3       elastic    0.857      0.104       0.022
4  RandomForest    0.807      0.176       0.037
5  DecisionTree    0.736      0.198       0.041
6           SVR    0.660      0.192       0.044
7       XGBoost   -2.086      0.515       0.126


# **With Sentiment**

In [20]:
import pandas as pd
from google.colab import drive
# Connecting to my google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
df_tesla_final = pd.read_csv('/content/drive/MyDrive/606 project/df_tesla_final_year.csv')

In [42]:
df_tesla_final.head(3)

,date,Open,High,Low,Close,Volume,mean_negative,count_negative,mean_positive,count_positive,mean_neutral,count_neutral,overall_mean_score,Next Day Open
0,2023-01-03,118.470001,118.800003,104.639999,108.099998,231402800,0.478478,713,0.526242,1087,0.000283,849,0.087063,109.110001
1,2023-01-04,109.110001,114.589996,107.519997,113.639999,180389000,0.484042,701,0.504059,994,0.000109,843,0.063684,110.510002
2,2023-01-05,110.510002,111.750000,107.160004,110.339996,157986300,0.464406,531,0.505971,818,0.000118,725,0.080617,103.000000


In [23]:
# df_tesla_final = df_tesla_final.iloc[::-1]

In [24]:
df_tesla_final.drop(columns=['Adj Close'], inplace=True)

## Splitting the columns for target variable

In [25]:
X= df_tesla_final.drop(columns=['Next Day Open'])
y=df_tesla_final['Next Day Open']

In [26]:
y

0      109.110001
1      110.510002
2      103.000000
3      118.959999
4      121.070000
          ...    
274    188.179993
275    189.000000
276    190.179993
277    192.110001
278    183.990005
Name: Next Day Open, Length: 279, dtype: float64

## Scaling

In [27]:
# Importing libraries
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
dates = X['date']
X = X.drop(columns=['date'])

In [29]:
# Define the column transformer
ct = make_column_transformer(
    (make_pipeline(KNNImputer(), StandardScaler()), slice(0, df_tesla_final.shape[1])),  # Apply to all columns
    remainder="passthrough"
)

# Fit and transform the data
scaled_data_X = ct.fit_transform(X)
scaled_X = pd.DataFrame(scaled_data_X, columns=X.columns)

scaler_y = StandardScaler()
scaled_y = scaler_y.fit_transform(y.values.reshape(-1, 1))

## Spliting the data into Train,Test

In [30]:
# Determine the index to split the data linearly
split_index = int(len(X) * 0.9)

# Split the data into training and testing sets
X_train = scaled_data_X[:split_index]
X_test = scaled_data_X[split_index:]
y_train = scaled_y[:split_index]
y_test = scaled_y[split_index:]

## Modeling with sentiment

In [31]:
# Importing librarues
from sklearn.linear_model import (LinearRegression, ElasticNetCV, RidgeCV)
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline

In [32]:
# Collecting the different type of estimators to select best regression model
estimators = {
    "lr": LinearRegression(),
    "lassocv": LassoCV(alphas=np.logspace(-5, -1, 100)),
    "ridge": RidgeCV(alphas=np.logspace(-5, -1, 100)),
    "elastic": ElasticNetCV(alphas=np.logspace(-5, 1, 100), l1_ratio=[.1, .5, .7, .9, .95, .99, 1]),
}

In [33]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
import pandas as pd

# Initialize an empty dictionary to store results
# results_dict = {'Estimator': [], 'R2 Mean': [], 'R2 Std': [], 'RMSE Mean': [], 'RMSE Std': [], 'NRMSE Mean': [], 'NRMSE Std': []}
results_dict = {'Estimator': [], 'R2 Mean': [], 'RMSE Mean': [], 'NRMSE Mean': []}

# Iterate over each estimator
for name, estimator in estimators.items():
    print(name)

    # Perform cross-validation
    cv_result = cross_validate(estimator, scaled_X, scaled_y, scoring=('r2', 'neg_root_mean_squared_error'), return_estimator=True, cv=10, verbose=2)

    # Extract R2 scores and RMSE scores
    r2_scores = cv_result['test_r2']
    rmse_scores = -cv_result['test_neg_root_mean_squared_error']

    # Calculate mean and standard deviation of scores
    r2_mean, r2_std = np.mean(r2_scores), np.std(r2_scores)
    rmse_mean, rmse_std = np.mean(rmse_scores), np.std(rmse_scores)

    # Calculate NRMSE
    y_true = scaled_y
    y_preds = -cv_result['test_neg_root_mean_squared_error']  # Negative RMSE from cross-validation
    mse = y_preds ** 2  # Convert negative RMSE to MSE
    nrmse = sqrt(mse.mean()) / (y_true.max() - y_true.min())  # NRMSE calculation

    # Append results to the dictionary
    results_dict['Estimator'].append(name)
    results_dict['R2 Mean'].append(round(r2_mean, 3))
    #results_dict['R2 Std'].append(round(r2_std, 3))
    results_dict['RMSE Mean'].append(round(rmse_mean, 3))
    #results_dict['RMSE Std'].append(round(rmse_std, 3))
    results_dict['NRMSE Mean'].append(round(nrmse, 3))


# Create DataFrame from the results dictionary
results_df = pd.DataFrame(results_dict)

# Print the DataFrame
print(results_df)


lr
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
lassocv
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END ........

In [34]:
pip install xgboost

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd

In [36]:
estimators = {
    "RandomForest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 5, 10, 15]
        }
    },
    "DecisionTree": {
        "model": DecisionTreeRegressor(),
        "params": {
            "max_depth": [None, 5, 10, 15]
        }
    },
    "SVR": {
        "model": SVR(),
        "params": {
            "kernel": ['linear', 'rbf'],
            "C": [0.1, 1, 10]
        }
    },
    "XGBoost": {
        "model": XGBRegressor(),
        "params": {
            "max_depth": [3, 5, 7],
            "learning_rate": [0.1, 0.01, 0.001],
            "n_estimators": [50, 100, 200]
        }
    }
}

In [37]:
for name, estimator in estimators.items():
    print(name)
    grid_search = GridSearchCV(estimator["model"], estimator["params"], scoring=('r2', 'neg_root_mean_squared_error'), cv=10, refit='r2', verbose=2)
    grid_search.fit(scaled_X, scaled_y)

    r2_scores = grid_search.cv_results_['mean_test_r2']
    rmse_scores = -grid_search.cv_results_['mean_test_neg_root_mean_squared_error']

    # Calculate NRMSE
    y_true = scaled_y  # Assuming scaled_y is the true target variable
    y_preds = -grid_search.cv_results_['mean_test_neg_root_mean_squared_error']  # Negative RMSE from cross-validation
    mse = y_preds ** 2  # Convert negative RMSE to MSE
    nrmse = sqrt(mse.mean()) / (y_true.max() - y_true.min())  # NRMSE calculation

    results_dict['Estimator'].append(name)
    results_dict['R2 Mean'].append(round(np.mean(r2_scores), 3))
    #results_dict['R2 Std'].append(round(np.std(r2_scores), 3))
    results_dict['RMSE Mean'].append(round(np.mean(rmse_scores), 3))
    #results_dict['RMSE Std'].append(round(np.std(rmse_scores), 3))
    results_dict['NRMSE Mean'].append(round(nrmse, 3))



RandomForest
Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.7s
[CV] END ...................max_d

In [38]:
results_df_with_sentiment = pd.DataFrame(results_dict)

In [39]:
results_df_without_sentiment

,Estimator,R2 Mean,RMSE Mean,NRMSE Mean
0,lr,0.856,0.104,0.022
1,lassocv,0.855,0.104,0.022
2,ridge,0.857,0.104,0.022
3,elastic,0.857,0.104,0.022
4,RandomForest,0.807,0.176,0.037
5,DecisionTree,0.736,0.198,0.041
6,SVR,0.660,0.192,0.044
7,XGBoost,-2.086,0.515,0.126


In [40]:
results_df_with_sentiment

,Estimator,R2 Mean,RMSE Mean,NRMSE Mean
0,lr,0.909,0.105,0.022
1,lassocv,0.908,0.106,0.022
2,ridge,0.910,0.105,0.022
3,elastic,0.908,0.106,0.022
4,RandomForest,0.796,0.181,0.038
5,DecisionTree,0.757,0.198,0.041
6,SVR,0.544,0.229,0.055
7,XGBoost,-2.093,0.519,0.126
